<a href="https://colab.research.google.com/github/jacobmrCuzzins/Machine-DeepLearning-/blob/main/TransferLearning_using_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classifying Images Using CNN and Transfer Learning
This Notebook contains the exploration of the effectiveness of training a CNN using the combination of transfer learning specifically VGG16 CNN.  



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Functions to work with CIFAR

The functions below help with access to the CIFAR-10 data the you have downloaded.

**NOTE:** This is different from previous notebooks to handle the input that the imagenet trained networks expect.

In [ ]:
import pickle
import numpy as np
import pandas as pd

def load_CIFAR_batch(filename, flatten=True, categorical=True):
    """ load single batch of cifar """
    with open(filename, 'rb') as f:
        datadict = pickle.load(f, encoding='bytes')
        X = datadict[b'data']
        X = X.reshape(10000, 3, 32, 32).transpose(0, 2, 3, 1).astype("float")
        if (flatten):
          X = X.reshape(10000, 3072)
        X = X.astype('float32')
        # need to keep the data in 0-255 range
        #X /= 255

        y = datadict[b'labels']
        y = np.array(y)
        if (categorical):
          y = pd.get_dummies(y).values

        return X, y

def load_CIFAR_meta(filename):
  with open(filename,'rb') as f:
    metadict = pickle.load(f, encoding='bytes')

    class_labels = [ val.decode() for val in metadict.get(b'label_names') ]
    return class_labels

def get_image(X, index, nchans=3, size=32):
  xi = np.copy(X[index,:])
  # now rescale for image display
  xi /= 255
  img = xi.reshape(size, size, nchans)
  return img

In [ ]:
flatten = False
categorical = True

# load only the first batch
#X1, y1 = load_CIFAR_batch('/content/drive/My Drive/cifar-10-batches-py/data_batch_1',flatten=flatten,categorical=categorical)

# load only the second batch
#X2, y2 = load_CIFAR_batch('/content/drive/My Drive/COMP2712/data/cifar-10-batches-py/data_batch_2',flatten=flatten,categorical=categorical)

# Load the first batch from CIFAR-10
X, y = load_CIFAR_batch('/content/drive/My Drive/cifar-10-batches-py/data_batch_1',flatten= False ,categorical=categorical)

# iterate over 2 to 5
for bi in range(2,6):
  # load the next data set 'bi'
  X2, y2 = load_CIFAR_batch('/content/drive/My Drive/cifar-10-batches-py/data_batch_{}'.format(bi),flatten=flatten,categorical=categorical)

  # concatenate/stack the dataest together
  X = np.vstack([X, X2])
  y = np.vstack([y, y2])

 #Load the test data into test variables
Xtest, ytest = load_CIFAR_batch('/content/drive/My Drive/cifar-10-batches-py/test_batch', flatten=False)

# Create 1d version for testing accuracy
ytrain_1d = np.argmax(y, axis=1)
ytest_1d = np.argmax(ytest, axis=1)

print('We have {} instances of images'.format(y.shape[0]))


We have 50000 instances of images


The number of instances/examples for all the different classes.  There are 10 different classes.

In [ ]:
[np.sum(np.argmax(y, axis=1) == i) for i in range(0,10)]

[5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000]

The labels for the classes are stored in the `batches.meta` file

In [ ]:
class_labels = load_CIFAR_meta('/content/drive/My Drive/cifar-10-batches-py/batches.meta')
print(class_labels)

['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


**START WITH THE ARNIE!**

Built the Base Model using VGG16 (Arnie) and added a custom classification layer (The Mr Bean)

In [ ]:
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras import layers

(X_train, y_train), (X_test, y_test) = cifar10.load_data() #Issues trying to load the dataset from my drive so imported from Keras datasets

# Preprocess data/Normalization
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# One-hot encode labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Load pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3)) #THE ARNIE!!

base_model.trainable = False #Make the base model Untrainable
# Create a new model on top of the pre-trained base model

model = Sequential([ # Custom classification layer #THE Mr Bean!!!
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.2),     # Included a dropout layer for regularization
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()



Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten_2 (Flatten)         (None, 512)               0         
                                                                 
 dense_4 (Dense)             (None, 256)               131328    
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_5 (Dense)             (None, 10)                2570      
                                                                 
Total params: 14848586 (56.64 MB)
Trainable params: 133898 (523.04 KB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


Trained the model!

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

Epoch 1/5
1563/1563 [==============================] - 26s 14ms/step - loss: 1.7796 - accuracy: 0.2940 - val_loss: 1.4706 - val_accuracy: 0.4175
Epoch 2/5
1563/1563 [==============================] - 22s 14ms/step - loss: 1.3049 - accuracy: 0.5068 - val_loss: 1.1658 - val_accuracy: 0.5629
Epoch 3/5
1563/1563 [==============================] - 21s 13ms/step - loss: 1.0201 - accuracy: 0.6342 - val_loss: 0.9358 - val_accuracy: 0.6596
Epoch 4/5
1563/1563 [==============================] - 22s 14ms/step - loss: 0.8382 - accuracy: 0.7073 - val_loss: 0.9272 - val_accuracy: 0.6911
Epoch 5/5
1563/1563 [==============================] - 23s 14ms/step - loss: 0.7396 - accuracy: 0.7442 - val_loss: 0.8499 - val_accuracy: 0.7139
Test Accuracy: 71.39%


**ENABLE THE BUFF DOGE!**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

base_model.trainable = True  #ENABLE THE BUFF DOGE!!
model.summary()

learning_rate = 1e-5

model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=learning_rate),
    loss=keras.losses.categorical_crossentropy,
    metrics=(['accuracy'])
)

epochs = 10 #Extend epochs to 10 allow for longer training
model.fit(X_train,y_train, epochs=epochs)
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten_4 (Flatten)         (None, 512)               0         
                                                                 
 dense_12 (Dense)            (None, 256)               131328    
                                                                 
 dropout_5 (Dropout)         (None, 256)               0         
                                                                 
 dense_13 (Dense)            (None, 10)                2570      
                                                                 
Total params: 14848586 (56.64 MB)
Trainable params: 14848586 (56.64 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
1

**Kfold Evaluation Allow the model to train for longer averaging the scores**

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

# Assuming you have your features in X and labels in y

# Initialize a KFold object with 3 splits
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Initialize a list to store accuracy scores
accuracy_scores = []

for fold, (train_index, test_index) in enumerate(kf.split(X, y), 1):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Assuming you have trained your model and obtained predictions
    model.fit(X_train, y_train, epochs=10, verbose=1)
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)

    # Calculate accuracy for this fold
    accuracy = accuracy_score(y_true_classes, y_pred_classes)
    accuracy_scores.append(accuracy)

    # Print metrics for this fold
    print(f"Fold {fold} - Accuracy: {accuracy * 100:.2f}%")

# Calculate the average accuracy over all folds
average_accuracy = np.mean(accuracy_scores) * 100
print(f"Average Accuracy: {average_accuracy:.2f}%")


Epoch 1/10
1250/1250 [==============================] - 16s 12ms/step - loss: 9.9997 - accuracy: 0.6451
Epoch 2/10
1250/1250 [==============================] - 15s 12ms/step - loss: 1.9810 - accuracy: 0.6392
Epoch 3/10
1250/1250 [==============================] - 14s 11ms/step - loss: 1.2458 - accuracy: 0.6424
Epoch 4/10
1250/1250 [==============================] - 16s 12ms/step - loss: 1.0157 - accuracy: 0.6809
Epoch 5/10
1250/1250 [==============================] - 14s 12ms/step - loss: 0.8828 - accuracy: 0.7164
Epoch 6/10
1250/1250 [==============================] - 14s 11ms/step - loss: 0.7878 - accuracy: 0.7451
Epoch 7/10
1250/1250 [==============================] - 14s 12ms/step - loss: 0.7095 - accuracy: 0.7667
Epoch 8/10
1250/1250 [==============================] - 14s 11ms/step - loss: 0.6494 - accuracy: 0.7851
Epoch 9/10
1250/1250 [==============================] - 15s 12ms/step - loss: 0.6033 - accuracy: 0.7969
Epoch 10/10
313/313 [==============================] - 1s 4ms/st